In [1]:
%pip install --upgrade jupyter ipywidgets
%pip install torch fairscale tiktoken==0.4.0 fair blobfile datasets mwparserfromhell

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging
import subprocess
import torch
import torch.distributed as dist
from common import save_to_disk, load_from_disk, save_layer_state_dict, load_layer_state_dict, model_args, tokenizer
from tokenizer import Tokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader, IterableDataset
import os
import time
from tqdm import tqdm
from model import TransformerBlock, VocabParallelEmbedding, ColumnParallelLinear, RMSNorm, Transformer, precompute_freqs_cis
from fairscale.nn.model_parallel.initialize import initialize_model_parallel, model_parallel_is_initialized

class StreamingWikipediaDataset(IterableDataset):
    def __init__(self, texts, tokenizer, seq_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.seq_len = seq_len

    def parse(self, text):
        tokens = self.tokenizer.encode(text, bos=True, eos=True)
        tokens = tokens[:self.seq_len] + [self.tokenizer.pad_id] * (self.seq_len - len(tokens))
        return torch.tensor(tokens)

    def __iter__(self):
        for item in self.texts:
            yield self.parse(item['text'])

def wait_for_file(filename, timeout=30):
    start_time = time.time()
    while not os.path.exists(filename):
        if time.time() - start_time > timeout:
            raise TimeoutError(f"Timeout waiting for {filename}")
        time.sleep(0.1)

def initialize_layers(model_args):
    # Initialize and save state dicts for all layers
    for layer_idx in range(model_args.n_layers):
        layer = TransformerBlock(layer_idx, model_args)
        save_layer_state_dict(layer.state_dict(), f"data/layer_{layer_idx}.pt")
    
    # Save the state dict for the embedding and output layers
    embedding = VocabParallelEmbedding(model_args.vocab_size, model_args.dim)
    save_layer_state_dict(embedding.state_dict(), "data/embedding.pt")
    
    output = ColumnParallelLinear(model_args.dim, model_args.vocab_size, bias=False)
    save_layer_state_dict(output.state_dict(), "data/output.pt")

    freqs_cis = precompute_freqs_cis(
        model_args.dim // model_args.n_heads,
        model_args.max_seq_len * 2,
        model_args.rope_theta,
    )
    save_to_disk(freqs_cis, "data/freqs_cis.pt")

def main():
    # Set environment variables for distributed training
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    os.environ['WORLD_SIZE'] = '1'
    os.environ['RANK'] = '0'
    
    # Initialize distributed environment
    dist.init_process_group(backend='nccl')
    initialize_model_parallel(model_parallel_size_=1)
    
    # Load the dataset using streaming
    dataset = load_dataset("wikipedia", language="en", date="20240401", split='train', streaming=True, trust_remote_code=True)
    
    # Prepare the dataset and dataloader
    seq_len = 2048
    wiki_dataset = StreamingWikipediaDataset(dataset, tokenizer, seq_len)
    dataloader = DataLoader(wiki_dataset, batch_size=1, shuffle=False, num_workers=0)

    num_epochs = 3
    learning_rate = 1e-4
    beta1 = 0.9
    beta2 = 0.999
    epsilon = 1e-8
    weight_decay = 1e-2

    # Initialize layer state dictionaries
    initialize_layers(model_args)

    for epoch in range(num_epochs):
        epoch_loss = 0

        for step, batch in enumerate(tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}")):
            try:
                process_batch(batch, tokenizer, epoch_loss, learning_rate, beta1, beta2, epsilon, weight_decay, step + 1)
            except Exception as e:
                logging.error(f"Error processing batch at step {step}: {e}")
                continue 

        print(f"Epoch {epoch + 1} Loss: {epoch_loss / len(dataloader)}")
        
        dist.destroy_process_group()

def process_batch(batch, tokenizer, epoch_loss, learning_rate, beta1, beta2, epsilon, weight_decay, t):
    try:
        batch = batch.to('cpu')
        save_to_disk(batch, "data/batch.pt")
        run_command(f"python3 worker.py --task embed --batch data/batch.pt --embedding_file data/embedding.pt --inputs data/inputs.pt")
        wait_for_file("data/inputs.pt")
        inputs = load_from_disk("data/inputs.pt")

        freqs_cis = load_from_disk("data/freqs_cis.pt")
        save_to_disk(freqs_cis, "data/freqs_cis.pt")

        for layer_idx in range(model_args.n_layers):
            inputs_file = f"data/inputs_layer_{layer_idx}.pt"
            logits_file = f"data/logits_layer_{layer_idx}.pt"
            state_dict_file = f"data/layer_{layer_idx}.pt"
            save_to_disk(inputs, inputs_file)
            run_command(f"python3 worker.py --task forward --layer_idx {layer_idx} --inputs {inputs_file} --state_dict {state_dict_file} --freqs_cis data/freqs_cis.pt --logits_file {logits_file}")
            wait_for_file(logits_file)
            inputs = load_from_disk(logits_file)

        norm = RMSNorm(model_args.dim, eps=model_args.norm_eps)
        inputs = norm(inputs)
        save_to_disk(inputs, "data/final_inputs.pt")

        run_command(f"python3 worker.py --task final_logits --inputs data/final_inputs.pt --state_dict data/output.pt --logits_file data/logits.pt")
        wait_for_file("data/logits.pt")
        logits = load_from_disk("data/logits.pt")
        
        save_to_disk(batch, "data/targets.pt")
        save_to_disk(logits, "data/logits.pt")
        run_command(f"python3 worker.py --task loss --logits data/logits.pt --targets data/targets.pt --loss_file data/loss.pt --logits_grad_file data/logits_grad.pt")
        wait_for_file("data/loss.pt")
        wait_for_file("data/logits_grad.pt")
        
        loss = load_from_disk("data/loss.pt")
        print(f"Loss: {loss:.4f}")
        epoch_loss += loss
        logits_grad = load_from_disk("data/logits_grad.pt")
        
        # Backpropagate through the output layer
        save_to_disk(logits_grad, "data/final_error.pt")
        run_command(f"python3 worker.py --task final_logits_backward --error data/final_error.pt --inputs data/final_inputs.pt --state_dict data/output.pt --error_output_file data/error_output.pt")
        wait_for_file("data/error_output.pt")
        final_error, output_grads = load_from_disk("data/error_output.pt")
        save_to_disk(output_grads, "data/output_grads.pt")
        run_command(f"python3 worker.py --task apply_adamw --layer_idx -1 --grads data/output_grads.pt --learning_rate {learning_rate} --beta1 {beta1} --beta2 {beta2} --epsilon {epsilon} --weight_decay {weight_decay} --t {t}")

        error = final_error[0]  # Use the input gradient for the next backward pass
        for layer_idx in reversed(range(model_args.n_layers)):
            inputs_file = f"data/inputs_layer_{layer_idx}.pt"
            error_output_file = f"data/error_layer_{layer_idx}.pt"
            state_dict_file = f"data/layer_{layer_idx}.pt"
            save_to_disk(error, "data/error_for_layer.pt")
            run_command(f"python3 worker.py --task backward --layer_idx {layer_idx} --error data/error_for_layer.pt --inputs {inputs_file} --state_dict {state_dict_file} --error_output_file {error_output_file}")
            wait_for_file(error_output_file)
            error_grads = load_from_disk(error_output_file)
            if error_grads is None or len(error_grads) != 2:
                raise ValueError(f"Failed to load error and grads from {error_output_file}")
            error, grads = error_grads
            error = error.view(1, -1, model_args.dim)  # Reshape to match the output shape of the previous layer
            save_to_disk(grads, f"data/grads_layer_{layer_idx}.pt")
            run_command(f"python3 worker.py --task apply_adamw --layer_idx {layer_idx} --grads data/grads_layer_{layer_idx}.pt --learning_rate {learning_rate} --beta1 {beta1} --beta2 {beta2} --epsilon {epsilon} --weight_decay {weight_decay} --t {t}")

        # Backpropagate through the embedding layer
        error = error.view(batch.size(0), -1, model_args.dim)  # Reshape to match the embedding layer output
        save_to_disk(error, "data/embedding_error.pt")
        run_command(f"python3 worker.py --task embed_backward --error data/embedding_error.pt --batch data/batch.pt --embedding_file data/embedding.pt --error_output_file data/embedding_grads.pt")
        wait_for_file("data/embedding_grads.pt")
        embedding_grads = load_from_disk("data/embedding_grads.pt")
        save_to_disk(embedding_grads, "data/embedding_grads.pt")
        run_command(f"python3 worker.py --task apply_adamw --layer_idx -2 --grads data/embedding_grads.pt --learning_rate {learning_rate} --beta1 {beta1} --beta2 {beta2} --epsilon {epsilon} --weight_decay {weight_decay} --t {t}")

    except Exception as e:
        logging.error(f"Failed to process and save batch: {e}")
        raise

def run_command(command):
    try:
        result = subprocess.run(command, shell=True, text=True, capture_output=True, check=True)
        logging.info(result.stdout)
    except subprocess.CalledProcessError as e:
        logging.error(f"Command '{command}' failed with error: {e.stderr}")
        raise

if __name__ == "__main__":
    main()


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Layer state dict saved to data/layer_0.pt
Layer state dict saved to data/layer_1.pt
Layer state dict saved to data/layer_2.pt
Layer state dict saved to data/layer_3.pt
Layer state dict saved to data/embedding.pt
Layer state dict saved to data/output.pt
Saved to data/freqs_cis.pt


Epoch 1/3: 0it [00:00, ?it/s]

Extracting content from https://dumps.wikimedia.org/enwiki/20240401/enwiki-20240401-pages-articles-multistream1.xml-p1p41242.bz2
Saved to data/batch.pt
Loaded from data/inputs.pt
Loaded from data/freqs_cis.pt
Saved to data/freqs_cis.pt
Saved to data/inputs_layer_0.pt
Loaded from data/logits_layer_0.pt
Saved to data/inputs_layer_1.pt
Loaded from data/logits_layer_1.pt
Saved to data/inputs_layer_2.pt
Loaded from data/logits_layer_2.pt
Saved to data/inputs_layer_3.pt
Loaded from data/logits_layer_3.pt
Saved to data/final_inputs.pt


ERROR:root:Command 'python3 worker.py --task norm --inputs data/final_inputs.pt --norm_file data/norm.pt --normed_output data/normed_output.pt' failed with error: Traceback (most recent call last):
  File "/home/user/Coding/ritser/magnum/spl/worker.py", line 238, in <module>
    norm_task(args.inputs, args.norm_file, args.normed_output)
  File "/home/user/Coding/ritser/magnum/spl/worker.py", line 186, in norm_task
    norm.load_state_dict(load_layer_state_dict(norm_file))
  File "/home/user/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py", line 2104, in load_state_dict
    raise TypeError(f"Expected state_dict to be dict-like, got {type(state_dict)}.")
TypeError: Expected state_dict to be dict-like, got <class 'NoneType'>.

ERROR:root:Failed to process and save batch: Command 'python3 worker.py --task norm --inputs data/final_inputs.pt --norm_file data/norm.pt --normed_output data/normed_output.pt' returned non-zero exit status 1.
ERROR:root:Error processing batch at 

Saved to data/batch.pt
Loaded from data/inputs.pt
Loaded from data/freqs_cis.pt
Saved to data/freqs_cis.pt
Saved to data/inputs_layer_0.pt
Loaded from data/logits_layer_0.pt
Saved to data/inputs_layer_1.pt
Loaded from data/logits_layer_1.pt
Saved to data/inputs_layer_2.pt
Loaded from data/logits_layer_2.pt
Saved to data/inputs_layer_3.pt
Loaded from data/logits_layer_3.pt
Saved to data/final_inputs.pt


...

Saved to data/batch.pt
Loaded from data/inputs.pt
Loaded from data/freqs_cis.pt
Saved to data/freqs_cis.pt
Saved to data/inputs_layer_0.pt
Loaded from data/logits_layer_0.pt
Saved to data/inputs_layer_1.pt
Loaded from data/logits_layer_1.pt
Saved to data/inputs_layer_2.pt
Loaded from data/logits_layer_2.pt
Saved to data/inputs_layer_3.pt
Loaded from data/logits_layer_3.pt
Saved to data/final_inputs.pt


ERROR:root:Command 'python3 worker.py --task norm --inputs data/final_inputs.pt --norm_file data/norm.pt --normed_output data/normed_output.pt' failed with error: Traceback (most recent call last):
  File "/home/user/Coding/ritser/magnum/spl/worker.py", line 238, in <module>
    norm_task(args.inputs, args.norm_file, args.normed_output)
  File "/home/user/Coding/ritser/magnum/spl/worker.py", line 186, in norm_task
    norm.load_state_dict(load_layer_state_dict(norm_file))
  File "/home/user/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py", line 2104, in load_state_dict
    raise TypeError(f"Expected state_dict to be dict-like, got {type(state_dict)}.")
TypeError: Expected state_dict to be dict-like, got <class 'NoneType'>.

ERROR:root:Failed to process and save batch: Command 'python3 worker.py --task norm --inputs data/final_inputs.pt --norm_file data/norm.pt --normed_output data/normed_output.pt' returned non-zero exit status 1.
ERROR:root:Error processing batch at 

Saved to data/batch.pt
Loaded from data/inputs.pt
Loaded from data/freqs_cis.pt
Saved to data/freqs_cis.pt
Saved to data/inputs_layer_0.pt
Loaded from data/logits_layer_0.pt
Saved to data/inputs_layer_1.pt
Loaded from data/logits_layer_1.pt
Saved to data/inputs_layer_2.pt
Loaded from data/logits_layer_2.pt
Saved to data/inputs_layer_3.pt
Loaded from data/logits_layer_3.pt
Saved to data/final_inputs.pt


ERROR:root:Command 'python3 worker.py --task norm --inputs data/final_inputs.pt --norm_file data/norm.pt --normed_output data/normed_output.pt' failed with error: Traceback (most recent call last):
  File "/home/user/Coding/ritser/magnum/spl/worker.py", line 238, in <module>
    norm_task(args.inputs, args.norm_file, args.normed_output)
  File "/home/user/Coding/ritser/magnum/spl/worker.py", line 186, in norm_task
    norm.load_state_dict(load_layer_state_dict(norm_file))
  File "/home/user/anaconda3/lib/python3.11/site-packages/torch/nn/modules/module.py", line 2104, in load_state_dict
    raise TypeError(f"Expected state_dict to be dict-like, got {type(state_dict)}.")
TypeError: Expected state_dict to be dict-like, got <class 'NoneType'>.

ERROR:root:Failed to process and save batch: Command 'python3 worker.py --task norm --inputs data/final_inputs.pt --norm_file data/norm.pt --normed_output data/normed_output.pt' returned non-zero exit status 1.
ERROR:root:Error processing batch at 

Saved to data/batch.pt
Loaded from data/inputs.pt
Loaded from data/freqs_cis.pt
Saved to data/freqs_cis.pt
Saved to data/inputs_layer_0.pt
Loaded from data/logits_layer_0.pt
Saved to data/inputs_layer_1.pt
Loaded from data/logits_layer_1.pt
Saved to data/inputs_layer_2.pt
Loaded from data/logits_layer_2.pt
Saved to data/inputs_layer_3.pt
